In [1]:
import pickle
import pandas as pd
import numpy as np
import streamlit as st
from model import ModelClinicValue
import corporate_wellness_app
import school_outreach_app
from model_forecasting import ModelForecastPerformance
from cashflow_plot import ModelCashflow
from streamlit_extras.stateful_button import button
import structured_saving as structured_saving_app
import os
    


In [2]:
    
clinic_data_set = pickle.load(open('clinic_value_set.pkl', 'rb'))
    
    
selected_clinic_set = list(clinic_data_set.keys())

In [3]:
# SAVE INTO PKL

file_name = 'clinic_projected_cashflow_set.pkl'


# Initialize or load existing dictionary
if os.path.exists(file_name):
    # Load existing data
    with open(file_name, 'rb') as f:
        clinic_projected_cashflow_set = pickle.load(f)
else:
    # Start with an empty dictionary
    clinic_projected_cashflow_set = {}

In [7]:

        
        
        
for selected_clinic in selected_clinic_set:
            
    clinic_data = clinic_data_set[selected_clinic]

    clinic_value = clinic_data['Clinic Valuation Adjusted']

    model = ModelForecastPerformance(clinic_data)

    
    percentage_borrow = 50
    upfront_payment = 20
                    

    interest_rate = 12
    loan_term = 5
    buying_year = 2025
             
    acquiring_funding = "Borrow"       
                    
    borrowed = (clinic_value * (percentage_borrow/100))
    principal = borrowed - ((upfront_payment/100) * borrowed)
    amortization_df, monthly_payment, total_principal, total_interest = model.loan_amortization_schedule(principal, interest_rate, loan_term, start_year=buying_year)
                
    sliced_amortization_df = amortization_df[['Period', 'Monthly Payment']]
    sliced_amortization_df = sliced_amortization_df.rename(columns={'Monthly Payment':"Expense"})
    sliced_amortization_df['Revenue'] = 0
                

    period_forecast = 12

    start_year = 2025
        

    number_of_months = model.total_days_from_start(period_forecast, start_year=start_year) 
    model_cashflow = ModelCashflow()
    forecast_df = model.forecast_revenue_expenses(period_forecast, start_month=1)
    indirect_expense = model.forecast_indirect_cost(period_forecast, start_year=start_year)
            
            

    treatment_details = pd.read_csv("treatment_with_details.csv")
    item_code_df = pd.read_csv("dummy_clinic_model\cleaned_item_code.csv")
            
    
    forecast_df_with_treatments = model.generate_forecast_item_code_by_profit(item_code_df, forecast_df, patient_pool=clinic_data['Patient Pool'], start_year=start_year)
            

            
    forecast_df = forecast_df_with_treatments[['Period', 'Revenue', 'Expense']].groupby('Period').sum().reset_index()


    sliced_amortization_df = sliced_amortization_df[:period_forecast] if acquiring_funding == "Borrow" else None
        
    equipment_df = model.equipment_to_cashflow(clinic_data['Equipment Life'], period_forecast, start_year=start_year)


    fitout_df = model.fitout_to_cashflow(clinic_data['Fitout Value'], clinic_data['Last Fitout Year'] ,period_forecast, start_year=start_year)


    

            
    clinic_value = clinic_data['Clinic Valuation Adjusted']
    clinic_ebit_multiple = clinic_data['EBIT Multiple']


    current_ebit = clinic_data['EBIT']
    current_ebit_ratio = clinic_data['EBIT Ratio']
    current_growth = clinic_data['Net Sales Growth']
    current_net_sales_relative_variation = clinic_data['Relative Variation of Net Sales']
            



        
    MA_forecast_df = model.forecast_revenue_expenses(3) # instead of using actual data, this method forecast three months as basis of most recent months of historical data 
                
    MA_forecast_df = model.forecast_revenue_expenses_MA(MA_forecast_df, period_forecast, 3, start_month=1)
                
                
                
                # MA_forecast_df_with_treatments = model.generate_forecast_treatment_df_by_profit(treatment_details, MA_forecast_df)
    MA_forecast_df_with_treatments = model.generate_forecast_item_code_by_profit(item_code_df, MA_forecast_df, start_year=start_year)
    MA_forecast_df = MA_forecast_df_with_treatments[['Period', 'Revenue', 'Expense']].groupby('Period').sum().reset_index()

    MA_indirect_cost = model.forecast_indirect_cost(period_forecast, start_year=start_year)

                
                # model_cashflow.remove_all_companies()
                # model_cashflow.add_company_data("Gross Profit", MA_forecast_df)
                # model_cashflow.add_company_data("Debt Repayment", sliced_amortization_df) if acquiring_funding == "Borrow" else None
                # model_cashflow.add_company_data("Indirect Expense", MA_indirect_cost)
                # model_cashflow.add_company_data("Equipment Procurement", equipment_df)
                # model_cashflow.add_company_data("Fit Out", fitout_df)
                
                # MA_forecast_linechart_daily = model_cashflow.cashflow_plot(number_of_months, start_date=generate_datetime(start_year=start_year))
                # MA_forecast_linechart_weekly = model_cashflow.cashflow_plot(number_of_months, granularity='weekly', start_date=generate_datetime(start_year=start_year))
                # MA_forecast_linechart_monthly = model_cashflow.cashflow_plot(number_of_months, granularity='monthly', start_date=generate_datetime(start_year=start_year))
                
            
            # current_ebit_after_12_months = current_ebit * (1+clinic_data['Net Sales Growth'])


            # potential_ebit_after_12_months = current_ebit_after_12_months
            
            
            
    ID = selected_clinic
            
    
    clinic_projected_cashflow_set[ID] = {
        "Approach 1": 
            {"Gross Profit": forecast_df,
            "Debt Repayment": sliced_amortization_df if acquiring_funding == "Borrow" else None,
            "Indirect Expense": indirect_expense,
            "Equipment Procurement": equipment_df,
            "Fit Out": fitout_df},
        "Approach 2": 
            {"Gross Profit": MA_forecast_df,
            "Debt Repayment": sliced_amortization_df if acquiring_funding == "Borrow" else None,
            "Indirect Expense": MA_indirect_cost,
            "Equipment Procurement": equipment_df,
            "Fit Out": fitout_df}
    }
            

    



        

        # st.markdown("### Strategy to implement")

        # col1, col2 = st.columns(2)

        # with col1:
        #     corporate_wellness = st.checkbox("Corporate Wellness Program", value=False)
        #     structured_saving = st.checkbox("Structured Saving Plan", value=False)
        #     fair_credit = st.checkbox("Fair Credit Program", value=False)
            
        # with col2:
        #     with st.popover("details"):
        #         corporate_wellness_app.app()
                
        #     with st.popover("details"):
        #         structured_saving_app.app()
                
        # # col1, col2 = st.columns(2)
        
        # # with col1:
        # #     school_outreach = st.checkbox("School Outreach Program", value=False)
            
        # # with col2:
        # #     with st.popover("details"):
        # #         school_outreach_app.app()
            
        


        # if button("Calculate New Cash Flow", key="calculate-strategy"):
            
        #     corporate_wellness_df = None
        #     structured_saving_df = None
        #     fair_credit_df = None
            
        #     if corporate_wellness == True:
        #         corporate_wellness_df = pd.read_csv("corporate_cashflow_AUD.csv")[:period_forecast]
        #         corporate_wellness_df['Period'] = corporate_wellness_df['Period'].apply(lambda period: model.generate_date_from_month(int(period), method='first_day', start_year=start_year))
        #         clinic_projected_cashflow_set[selected_clinic]["Approach 1"]["Corporate Wellness"] = corporate_wellness_df
        #         clinic_projected_cashflow_set[selected_clinic]["Approach 2"]["Corporate Wellness"] = corporate_wellness_df
                
        #     if structured_saving == True:
        #         structured_saving_df = pd.read_csv("structured_saving_cashflow.csv")[:period_forecast]
        #         structured_saving_df['Period'] = structured_saving_df['Period'].apply(lambda period: model.generate_date_from_month(int(period), method='first_day', start_year=start_year))
        #         clinic_projected_cashflow_set[selected_clinic]["Approach 1"]["Structured Saving"] = structured_saving_df
        #         clinic_projected_cashflow_set[selected_clinic]["Approach 2"]["Structured Saving"] = structured_saving_df
            
        #     if fair_credit == True:
        #         fair_credit_df = pd.read_csv("fair_credit_cashflow.csv")[:period_forecast]
        #         fair_credit_df['Period'] = fair_credit_df['Period'].apply(lambda period: model.generate_date_from_month(int(period), method='first_day', start_year=start_year))
        #         clinic_projected_cashflow_set[selected_clinic]["Approach 1"]["Fair Credit"] = fair_credit_df
        #         clinic_projected_cashflow_set[selected_clinic]["Approach 2"]["Fair Credit"] = fair_credit_df

        #     model_cashflow.add_company_data("Corporate Wellness", corporate_wellness_df) if corporate_wellness_df is not None else None
        #     model_cashflow.add_company_data("Structured Saving", structured_saving_df) if structured_saving_df is not None else None
        #     model_cashflow.add_company_data("Fair Credit", fair_credit_df) if fair_credit_df is not None else None
        #     st.plotly_chart(model_cashflow.cashflow_plot(number_of_months, granularity='monthly', start_date=generate_datetime(start_year=start_year)))
            
            
        # st.write(clinic_projected_cashflow_set)

In [8]:
len(clinic_projected_cashflow_set)

50

In [9]:
# save into pkl
with open(file_name, 'wb') as f:
    pickle.dump(clinic_projected_cashflow_set, f)